# Spreadsheet

    Make a spreadsheet using pinkfish.  This is useful for developing trading strategies.
    It can also be used as a tool for buy and sell signals that you then manually execute.

In [1]:
import datetime

import matplotlib.pyplot as plt
import pandas as pd

from talib.abstract import *

import pinkfish as pf
import pinkfish.itable as itable

# Format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# Set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)
ts.tail()

,open,high,low,close,adj_close,volume
date,,,,,,
2025-01-30,603.96,606.60,600.72,605.04,605.04,39281300
2025-01-31,607.50,609.96,601.05,601.82,601.82,66671500
2025-02-03,592.67,600.29,590.49,597.77,597.77,65857200
2025-02-04,597.83,602.30,597.28,601.78,601.78,33457800
2025-02-05,600.64,604.37,598.58,604.22,604.22,26711561


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [5]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)
ts.head()

,open,high,low,close,adj_close,volume
date,,,,,,
1993-01-29,43.97,43.97,43.75,43.94,24.53,1003200
1993-02-01,43.97,44.25,43.97,44.25,24.70,480500
1993-02-02,44.22,44.38,44.12,44.34,24.75,201300
1993-02-03,44.41,44.84,44.38,44.81,25.01,529400
1993-02-04,44.97,45.09,44.47,45.00,25.12,531500


Add technical indicators

In [6]:
# Add 200 day MA.
ts['sma200'] = pf.SMA(ts, timeperiod=200)

# Add ATR.
ts['atr'] = ATR(ts, timeperiod=14)

# Add 5 day high, and 5 day low
ts['high5'] = pd.Series(ts.high).rolling(window=5).max()
ts['low5'] = pd.Series(ts.low).rolling(window=5).min()

# Add RSI, and 2-period cumulative RSI
ts['rsi2'] = RSI(ts, timeperiod=2)
ts['c2rsi2'] = pd.Series(ts.rsi2).rolling(window=2).sum()

# Add midpoint
ts['mp'] = (ts.high + ts.low) / 2

# Add 10 day SMA of midpoint
ts['sma10'] = pd.Series(ts.mp).rolling(window=10).mean()

# Add temporary rolling 10 day Standard Deviation of midpoint
ts['__sd__'] = pd.Series(ts.mp).rolling(window=10).std()

# Add standard deviation envelope or channel around midpoint
ts['upper'] = ts.sma10 + ts['__sd__']*2
ts['lower'] = ts.sma10 - ts['__sd__']*2

# Drop temporary columns.
ts.drop(columns=['__sd__'], inplace=True)

Finalize timeseries

In [7]:
ts, start = pf.finalize_timeseries(ts, start, dropna=True)
ts.tail()

,open,high,low,close,adj_close,volume,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,
2025-01-30,603.96,606.60,600.72,605.04,605.04,39281300,561.04,7.02,610.78,594.64,65.12,102.53,603.66,601.78,611.99,591.57
2025-01-31,607.50,609.96,601.05,601.82,601.82,66671500,561.53,7.15,609.96,594.64,34.59,99.71,605.51,603.07,611.18,594.95
2025-02-03,592.67,600.29,590.49,597.77,597.77,65857200,562.00,7.45,609.96,590.49,15.87,50.46,595.39,602.86,611.69,594.03
2025-02-04,597.83,602.30,597.28,601.78,601.78,33457800,562.51,7.28,609.96,590.49,59.39,75.26,599.79,602.75,611.72,593.79
2025-02-05,600.64,604.37,598.58,604.22,604.22,26711561,563.03,7.17,609.96,590.49,75.08,134.47,601.48,602.24,610.81,593.67


Select a smaller time from for use with itable

In [8]:
df = ts['2023-06-01':]
df.head()

,open,high,low,close,adj_close,volume,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,
2023-06-01,418.09,422.92,416.79,421.82,411.81,88865000,396.61,4.58,422.92,412.41,78.64,120.15,419.86,417.32,422.88,411.75
2023-06-02,424.50,428.74,423.95,427.92,417.76,91366700,396.60,4.74,428.74,415.25,92.76,171.40,426.35,418.23,426.20,410.27
2023-06-05,428.28,429.62,426.37,427.10,416.96,65460200,396.60,4.64,429.62,416.22,78.76,171.52,427.99,419.13,429.23,409.03
2023-06-06,426.67,428.58,425.99,428.03,417.87,64022200,396.60,4.49,429.62,416.22,84.18,162.94,427.28,419.97,431.30,408.64
2023-06-07,428.44,429.62,426.11,426.55,416.43,85373300,396.62,4.42,429.62,416.79,46.46,130.64,427.86,421.14,433.12,409.15


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [9]:
pt = itable.PrettyTable(
    df, tstyle=itable.TableStyle(theme='theme1'), header_row=True, rpt_header=20)

# pt = itable.PrettyTable(
#      df, tstyle=itable.TableStyle(theme='theme1'), header_row=True, rpt_header=20)

pt.update_col_header_style(
    format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(
    format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'].iloc[row] == pt.df['high'].iloc[row]) and \
       (pt.df['high5'].iloc[row] > pt.df['high'].iloc[row-1]):
        col = df.columns.get_loc('high5')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'].iloc[row] == pt.df['low'].iloc[row]) and \
       (pt.df['low5'].iloc[row] < pt.df['low'].iloc[row-1]):
        col = df.columns.get_loc('low5')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [10]:
pt

,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10,UPPER,LOWER
2023/06/01,418.09,422.92,416.79,421.82,411.81,88865000,396.61,4.58,422.92,412.41,78.64,120.15,419.86,417.32,422.88,411.75
2023/06/02,424.50,428.74,423.95,427.92,417.76,91366700,396.60,4.74,428.74,415.25,92.76,171.40,426.35,418.23,426.20,410.27
2023/06/05,428.28,429.62,426.37,427.10,416.96,65460200,396.60,4.64,429.62,416.22,78.76,171.52,427.99,419.13,429.23,409.03
2023/06/06,426.67,428.58,425.99,428.03,417.87,64022200,396.60,4.49,429.62,416.22,84.18,162.94,427.28,419.97,431.30,408.64
2023/06/07,428.44,429.62,426.11,426.55,416.43,85373300,396.62,4.42,429.62,416.79,46.46,130.64,427.86,421.14,433.12,409.15
2023/06/08,426.62,429.60,425.82,429.13,418.94,61952800,396.70,4.38,429.62,423.95,79.10,125.57,427.71,422.77,433.19,412.36
2023/06/09,429.96,431.99,428.87,429.90,419.70,85742800,396.79,4.29,431.99,425.82,84.68,163.78,430.43,424.39,433.92,414.86
2023/06/12,430.92,433.88,430.17,433.80,423.50,76104300,396.89,4.26,433.88,425.82,95.86,180.54,432.03,425.79,435.28,416.30
2023/06/13,435.32,437.33,434.63,436.66,426.30,95899700,396.98,4.21,437.33,425.82,98.00,193.86,435.98,427.32,438.00,416.64
2023/06/14,437.01,439.06,433.59,437.18,426.80,100612100,397.13,4.30,439.06,425.82,98.32,196.32,436.32,429.18,438.86,419.50


In [12]:
pf.get_quote([symbol])

{'SPY': 604.219970703125}